![Linux Academy](la-logo.png)
<hr/>

<center><h1>PyTorch Simple CNN</h1></center>

![Sorting Lego bricks](./lego.jpg)

<center><h2>Lego Brick Sorting</h2></center>

# Introduction

In this Linux Academy hands-on lab, we take an introductory look at __PyTorch__ (with `PyTorch.nn`) and use it to make a simple convolutional neural network.

## PyTorch

![PyTorch Logo](./pytorch-logo.png)

PyTorch is an open source, deep learning platform that provides a seamless path from research prototyping to production deployment.

_(Source: https://pytorch.org)_

PyTorch is very powerful, and with great power... comes a steep learning curve. To help with this and other optimizations, ```pytorch.nn``` is a set of Modules (with a capital M; these are not the same as Python modules). They have prewritten, optimized code for many common ML tasks.

In this Linux Academy hands-on lab we use some of the ```pytorch.nn``` Modules to get us going faster.


# Scenario

We have bricks, lots of bricks, Lego bricks that is. And we need to get them sorted.

We have a collection of photos of different Lego bricks from different angles. We have 600 photos (really we do) and they are all labeled with the brick name.

Each photo has been processed by increasing the contrast, sharpening, removing the color, inverting the colors, and reducing its size.

|![Sample Lego brick photo](./sample-before.png)|![Sample Lego brick photo](./sample-after.png)|
|----------------------------------------|----------------------------------------|
| Sample before processing                | Sample after processing                |

In addition to this, we loaded all the images into a single data array for easier loading into an algorithm. If you're interested in how these images were collected and processed contact me through the Linux Academy Community Slack: ```@mike chambers```.

We're going to create a simple, deep learning, neural network classifier model. We will train the model using the photo data and see if it correctly predicts or infers the type of a brick from a supplied test image.

# How to Use This Lab

This is a follow-along lab. That is to say the code in this Jupyter Notebook should be complete, and you could simply execute the notebook to get a result. However, watch along with the video to learn more about what is happening in the code and then take the time to experiment with the code; make changes, break it, fix it, and learn!

# 1) The Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

Before we get started, let's take a moment to set the processor type that PyTorch will be using. PyTorch is optimized for running on GPUs. Since this Juypter Notebook server doesn't have access to GPUs, we need to tell PyTorch to use the CPU instead. That will be ok for what we are doing here.

In [ ]:
device = torch.device('cpu')

# 2) Load the data

## [NumPy]

First we load the data. This lab includes four files containing the data we want:

1. `lego-simple-lin-train-images.npy` - Training images, around 80% of the data collected.
2. `lego-simple-lin-train-labels.npy` - A list of integer labels that identifiy the classes of the training images.
3. `lego-simple-lin-test-images.npy`  - Testing images, around 20% of the data collected.
4. `lego-simple-lin-test-labels.npy`  - A list of integer labels that identifiy the classes of the testing images.

The data is stored in NumPy file format. We convert it to PyTorch tensors later.

In [ ]:
x_train = np.load('lego-simple-lin-train-images.npy').astype(np.float32)
y_train = np.load('lego-simple-lin-train-labels.npy')
x_valid = np.load('lego-simple-lin-test-images.npy').astype(np.float32)
y_valid = np.load('lego-simple-lin-test-labels.npy')

The label data we loaded are integer values (1,2,3). Let's get some human names for the data classes we're working with.

In [ ]:
# For humans:
class_names = ['2x3 Brick', '2x2 Brick', '1x3 Brick', '2x1 Brick', '1x1 Brick', '2x2 Macaroni', '2x2 Curved End', 'Cog 16 Tooth', '1x2 Handles', '1x2 Grill']

# Or the real Lego codes:
# class_names = ['3002', '3003', '3622', '3004', '3005', '3063', '47457', '94925', '3839a', '2412b']

The images we are using are 48 by 48 pixels. We set that here as a convenience, so if we want to try other images with this code, we change these values. For now, this works as is.

In [ ]:
image_width = 48
image_height = 48

After loading data, especially image data, it's good to visialize it. If nothing else, it helps us see we didn't mangle the data.

The following is a helper function to plot the image data.

In [ ]:
def plot_img(data, label):
    plt.figure()
    plt.imshow(data.reshape((image_width, image_height)))
    plt.colorbar()
    plt.xlabel(class_names[label])
    plt.show()

Next, we choose an image from our set and send it to the plotting function.

In [ ]:
img_number = 0
plot_img(x_train[img_number], y_train[img_number])

## [PyTorch]

We have the data loaded into NumPy arrays. Now we transfer the data into PyTourch tensors.

Tensors act like arrays but with extra capability. When we create the tensors, we pass the ```device``` object we created so PyTorch configures them for the appropriate processor architecture.

In [ ]:
x_train = torch.tensor(x_train).to(device)
y_train = torch.tensor(y_train).to(device)
x_valid = torch.tensor(x_valid).to(device)
y_valid = torch.tensor(y_valid).to(device)

Again, let's take a quick look at the data. PyTorch tensors work a lot like NumPy arrays. We can even use ```.shape``` to describe the data.

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

And the data itself, or at least a summary of it.

In [ ]:
x_train, x_valid

# Create the Model

Before we create the model, we set some hyperparameters that impact how the model learns.

In [ ]:
learning_rate = 0.1
epochs = 20
batch_size = 34

Now with the hyperparameters set, we can create some dataloaders. These dataloaders are `PyTorch.nn` objects that help with loading data into the model as it trains.

In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)

The next section of code creates the model itself. We are using the sequential model of `PyTorch.nn` to allow us to create our model quickly and logically.

We create `TorchModule` as a wrapper for a preprocessing layer and our output layer. We could use the same structure to make a layer in our network be anything we want.

The model itself is sequentially defined as a convolutional neural network (CNN). Each layer after the first is defined as the number of nodes in the previous layer plus the number of nodes in the current layer. Consider the following:

```
nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),
nn.ReLU(),
```

This defines a two-dimentional CNN layer connecting from 32 nodes in the previous layer, to 16 modes in this layer.

* ```kernel_size``` - The size of the 'kernel' that convolves over the image.
* ```stride``` - The step size the 'kernel' uses when convolving. 
* ```padding``` - A padding applied to the image so we don't lose information from the edge.

Finally, in the code block above, we add a rectified linear unit (ReLU) as an activation function.

In [ ]:
class TorchModule(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

def preprocess(x):
    return x.view(-1, 1, image_width, image_height).to(device)

model = nn.Sequential(
    TorchModule(preprocess),
    nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    TorchModule(lambda x: x.view(x.size(0), -1)),
)

Now we define one of the built-in optimization algorithms to use and a loss function to use.

In [ ]:
opt = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_func = F.cross_entropy

In the following block, we define some functions to help us with the tasks at hand.

In [ ]:
# loss_batch calculates the loss and is intended to be used during training.
# If no optimizer is being passed then we are in a validation phase and we miss it out.
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), len(xb)


# We are writing our own fit function.
# You can see the structure of the function going through the steps required to train.
# As we progress through the epochs we store the loss value so that we can summarise the 
# training process in a graph later.
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    history = []
    for epoch in range(epochs):
        
        # First we train:
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        # Then we evaluate:
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        history.insert(epoch,val_loss)
        print('.', end='')
        
    print('[Done]')
    return history

# Train

Now we set it all in motion. We train and then plot a graph of the loss function over the epochs. The loss function looks at the model prediction and the actual labeled data. It calculates the difference or loss. We want the loss to get lower as the training progresses.

In [ ]:
history = fit(epochs, model, loss_func, opt, train_dl, valid_dl)

plt.figure(figsize=(7, 4))
plt.plot(history)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

# Test

Let's test out the model. Choose one of the iamges from our validation set. _(Yes, you're right, we shouldn't test with an image from our validation set, but if it doesn't get this right, then we're in trouble.)_

In [ ]:
img_to_test = 0

First, what is this image? Let's check to see what the prediction should be.

In [ ]:
plot_img(x_valid[img_to_test].numpy(), y_valid[img_to_test].numpy())

Now, let's make the prediction. Did the model get it right?

In [ ]:
p = model(x_valid[img_to_test])

prediction = torch.argmax(p)
class_label = class_names[prediction.numpy()]

prediction, class_label

# Evaluate

If we're happy with the model up until this point, let's throw a lot of data at it. Let's make predictions from the whole validation set.  

To help display the output, here are a couple of helper functions.

In [ ]:
def plot_image(predictions_array, true_label, img):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img.reshape((image_width, image_height)), cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    # Print a label with 'predicted class', 'probability %', 'actual class'
    plt.xlabel("{} [{:2.0f}] ({})".format(class_names[predicted_label],
                                  np.max(predictions_array),
                                  class_names[true_label]),
                                  color=color)

# Function to display the prediction results in a graph:
def plot_value_array(predictions_array, true_label):
  plt.xticks(range(10))
  plt.yticks([])
  plot = plt.bar(range(10), predictions_array, color="#777777")
  predicted_label = np.argmax(predictions_array)
  plot[predicted_label].set_color('red')
  plot[true_label].set_color('green')

Make predictions from all the validation data.

In [ ]:
predictions = model(x_valid)

Display the results.

In [ ]:
num_rows = 8
num_cols = 2

num_images = num_rows*num_cols
plt.figure(figsize=(15, 16))

for i in range(num_images):
    ipred = predictions[i].detach().numpy()
    iimg = x_train[i].detach().numpy()
    ilab = y_valid[i].detach().numpy()
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(ipred, ilab, iimg)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(ipred, ilab)
plt.show()

That's the end of this Linux Academy hands-on lab. Thanks!